In [5]:
import pandas as pd
df_0 = pd.read_csv("./data/movie_list_0.csv")
df_1 = pd.read_csv("./data/movie_list_1.csv")
df_2 = pd.read_csv("./data/movie_list_2.csv")
df_3 = pd.read_csv("./data/movie_list_3.csv")

df_4 = pd.read_csv("./data/movie_list_4.csv")
df_5 = pd.read_csv("./data/movie_list_5.csv")
df_6 = pd.read_csv("./data/movie_list_6.csv")

df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True)
# df = pd.concat([df_0, df_1, df_2, df_3], ignore_index=True)

In [6]:
df = df.drop_duplicates()
df = df.reset_index(drop = True)

In [7]:
def to_dict(tu):
    if type(tu[0]) == str:
        d = {tu[0] : tu[1]}
    else:
        temp = [{x[0] : x[1]} for x in tu]
        d = {}
        for t in temp:
            d.update(t)
    return d

import ast

def to_ast(s):  # " "  빈공간
    try:
        result = ast.literal_eval(s)
        return False
    except: 
        return True

In [8]:
stream_index = df["스트리밍"].notna()
nan_index = df[stream_index][df["스트리밍"][stream_index].apply(lambda x : x.strip("[]")).apply(to_ast)].index
df["스트리밍"][nan_index] = np.nan

stream_index = df["구매"].notna()
nan_index = df[stream_index][df["구매"][stream_index].apply(lambda x : x.strip("[]")).apply(to_ast)].index
df["구매"][nan_index] = np.nan

stream_index = df["대여"].notna()
nan_index = df[stream_index][df["대여"][stream_index].apply(lambda x : x.strip("[]")).apply(to_ast)].index
df["대여"][nan_index] = np.nan

In [9]:
stream_index = df["스트리밍"].notna()
df["스트리밍"][stream_index] = df["스트리밍"][stream_index].apply(lambda x : x.strip("[]")).apply(lambda x : eval(x)).apply(to_dict)
stream_index = df["구매"].notna()
df["구매"][stream_index] = df["구매"][stream_index].apply(lambda x : x.strip("[]")).apply(lambda x : eval(x)).apply(to_dict)
stream_index = df["대여"].notna()
df["대여"][stream_index] = df["대여"][stream_index].apply(lambda x : x.strip("[]")).apply(lambda x : eval(x)).apply(to_dict)

In [10]:
df = df[['제목', '원제', '개봉일', '스트리밍', '대여', '구매', '평점', '장르', '재생 시간', '감독', '출연진', '시놉시스', '이미지']]

In [11]:
df["장르"] = df["장르"].apply(lambda x : x.strip("['']"))
df["장르"][df["장르"] == ""] = np.nan
not_index = df["장르"].notna()
df["장르"][not_index] = df["장르"][not_index].apply(lambda x : x.split(" , "))
df["장르"][not_index] = df["장르"][not_index]

In [12]:
# not_index = df["감독"].notna()
# df["감독"][not_index] = df["감독"][not_index].apply(lambda x: x.strip("['']"))
# df["감독"][not_index] = df["감독"][not_index].apply(lambda x : x.split(" , "))

In [13]:
df["평점"][df["평점"] == "[]"] = np.nan
df["평점"][df["평점"] == "['']"] = np.nan
df["재생 시간"][df["재생 시간"] == "[]"] = np.nan
df["재생 시간"][df["재생 시간"] == "['']"] = np.nan
df["감독"][df["감독"] == "[]"] = np.nan
df["감독"][df["감독"] == "['']"] = np.nan
df["출연진"][df["출연진"] == "[]"] = np.nan
df["출연진"][df["출연진"] == "['']"] = np.nan

In [18]:
df["개봉일"] = df["개봉일"].apply(lambda x : x.strip("()"))
df["재생 시간"] = df["재생 시간"].apply(lambda x : x.strip("[]"))

In [19]:
df.to_csv("./data/movie_data.csv", index=False)

In [20]:
df_all = pd.read_csv("./data/movie_7000_end.csv")

In [21]:
df = pd.concat([df, df_all], ignore_index=True)

In [22]:
df.to_csv("./data/movie.csv", index=False)

# 추천 시스템

In [367]:
df = pd.read_csv("temp_mocie_data.csv")

In [368]:
df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
df["장르"] = df["장르"].apply(lambda x: eval(x))
df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

In [369]:
isna_index = df["평점"].isna()
df["평점"][isna_index] = "['']"
df["평점"] = df["평점"].apply(lambda x : eval(x))

In [370]:
def print_ls_1(x):
    if len(x) == 1:
        return 0
    else:
        return x[1].split()[0]
    
def print_ls_2(x):
    if len(x) == 1:
        return "0k"
    else:
        try:
            return x[1].split()[1].strip("()")
        except:
            return "0k"

In [371]:
df["평가수"] = df["평점"].apply(print_ls_2)
df["평점"] = df["평점"].apply(print_ls_1)

In [372]:
def test(x):
    return 

In [373]:
df["평가수"] = df["평가수"].apply(lambda x: str(int(x[:-1]) * 1000) if x[-1] == "m" else x[:-1])

In [374]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(df["장르"])
genre_mat.shape

(4000, 293)

In [375]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4000, 4000)
[[1.         0.21821789 0.21821789 ... 0.         0.         0.21821789]
 [0.21821789 1.         0.14285714 ... 0.52414242 0.52414242 0.14285714]]


In [376]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [377]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 20):
    title_movie = df[df["제목"] == title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [378]:
sim_movies = find_sim_movie(df, genre_sim_sorted_ind, "나 홀로 집에",20)

[[   0 1805 3535 2449 2792 2310 2153  586 2277 2966 3449 2470 1919 2832
  2600  624  636 3875  393 2898]]


In [379]:
sim_movies[["제목", "평점"]]

,제목,평점
0,나 홀로 집에,7.6
1805,솔드 아웃,5.7
3535,크리스마스 스타!,6.4
2449,대디 데이 케어,5.6
2792,102 달마시안,4.9
2310,빈,6.5
2153,홈 포 크리스마스,5.5
586,머펫 무비,7.6
2277,열두 명의 웬수들,5.9
2966,First Kid,5.3


In [380]:
df["평점"] = df["평점"].astype("float")
df["평가수"] = df["평가수"].astype("int")

In [381]:
C = df["평점"].mean()
m = df["평가수"].quantile(0.6)
print(f"C: {round(C,3)}, m: {round(m, 3)}")

C: 6.337, m: 43.0


In [382]:
def weighted_vote_average(record):
    v = record["평가수"]
    R = record["평점"]
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)

In [383]:
df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

In [384]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 20):
    title_movie = df[df["제목"] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:top_n]

In [385]:
sim_movies = find_sim_movie(df, genre_sim_sorted_ind, "나 홀로 집에",20)
sim_movies[["제목", "평점", "가중치평점"]]

,제목,평점,가중치평점
624,곰돌이 푸의 모험,7.6,6.894912
586,머펫 무비,7.6,6.894912
636,아리스토캣,7.1,6.867434
2310,빈,6.5,6.451446
393,애들이 줄었어요,6.4,6.385980
3535,크리스마스 스타!,6.4,6.348235
2600,키드,6.1,6.221526
2832,에어,5.5,6.206043
2153,홈 포 크리스마스,5.5,6.192465
2966,First Kid,5.3,6.141664


In [386]:
df["제목"][df["제목"].apply(lambda x: "나 홀로" in x)]

0                  나 홀로 집에
13      나 홀로 집에 2: 뉴욕을 헤매다
1884             나 홀로 집에 3
1918             나 홀로 집에 4
Name: 제목, dtype: object

# 난 나홀로 집에 2가 더 어울리는것 같다.

In [387]:
def sum_list(x):
    ls = []
    for i in x[:5]:
        ls.append("_".join(i.split(" ")))
    return " ".join(ls)

In [388]:
df["감독"][df["감독"].isna()] = "['']"
df["감독"] = df["감독"].apply(lambda x: eval(x))
df["감독"] = df["감독"].apply(sum_list)

In [389]:
df["출연진"][df["출연진"].isna()] = "['']"
df["출연진"] = df["출연진"].apply(lambda x: eval(x))
df["출연진"] = df["출연진"].apply(sum_list)

In [390]:
df["특징"] = df["장르"] + " " + df["감독"] + " " + df["출연진"]

In [391]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(df["특징"])
genre_mat.shape

(4000, 34620)

In [392]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4000, 4000)
[[1.         0.05057217 0.05564149 ... 0.         0.         0.05564149]
 [0.05057217 1.         0.04783649 ... 0.20851441 0.20851441 0.04783649]]


In [393]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [394]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 20):
    title_movie = df[df["제목"] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:top_n]

In [399]:
sim_movies = find_sim_movie(df, genre_sim_sorted_ind, "나 홀로 집에",20)
sim_movies[["제목", "감독", "장르", "출연진", "평점", "가중치평점"]]

,제목,감독,장르,출연진,평점,가중치평점
30,팀버튼의 크리스마스 악몽,Henry_Selick,판타지 애니메이션 가족,Chris_Sarandon Danny_Elfman Catherine_O'Hara K...,8.0,7.803053
55,비틀쥬스,Tim_Burton,판타지 코미디,Alec_Baldwin Geena_Davis Jeffrey_Jones Catheri...,7.5,7.346179
137,미세스 다웃파이어,Chris_Columbus,드라마 코미디 가족,Robin_Williams Sally_Field Pierce_Brosnan Lisa...,7.0,6.905027
586,머펫 무비,James_Frawley,코미디 가족,Jim_Henson Frank_Oz Jerry_Nelson Richard_Hunt ...,7.6,6.894912
13,나 홀로 집에 2: 뉴욕을 헤매다,Chris_Columbus,코미디 다큐멘터리 범죄 가족,Macaulay_Culkin Joe_Pesci Daniel_Stern Catheri...,6.8,6.746955
3986,어웨이 위 고,Sam_Mendes,코미디 드라마 로맨스,John_Krasinski Maya_Rudolph Carmen_Ejogo Cathe...,7.0,6.700086
3798,페넬로피,Mark_Palansky,판타지 로맨스 코미디 Made in Europe,Christina_Ricci James_McAvoy Catherine_O'Hara ...,6.7,6.551507
2310,빈,Mel_Smith,코미디 가족,Rowan_Atkinson Peter_MacNicol Pamela_Reed Tric...,6.5,6.451446
3535,크리스마스 스타!,Debbie_Isitt,코미디 가족,Martin_Freeman Ashley_Jensen Marc_Wootton Pam_...,6.4,6.348235
1545,소포야 꼼짝마,Charles_A._Nichols,애니메이션 코미디 가족,Pinto_Colvig,0.0,6.337400


In [396]:
df[df["제목"] == "해리 포터와 마법사의 돌"][["제목", "감독", "장르", "평점", "가중치평점"]]

,제목,감독,장르,평점,가중치평점
1735,해리 포터와 마법사의 돌,Chris_Columbus,판타지 액션 가족,7.6,7.528657


In [398]:
sim_movies = find_sim_movie(df, genre_sim_sorted_ind, "해리 포터와 마법사의 돌",20)
sim_movies[["제목", "감독", "장르", "출연진", "평점", "가중치평점"]]

,제목,감독,장르,출연진,평점,가중치평점
1806,해리 포터와 아즈카반의 죄수,Alfonso_Cuarón,판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Juli...,7.9,7.793852
3392,해리 포터와 불의 잔,Mike_Newell,판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Robb...,7.7,7.606552
3495,해리 포터와 혼혈 왕자,David_Yates,판타지 가족 액션,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,7.6,7.501467
3530,해리 포터와 불사조 기사단,David_Yates,스릴러 액션 판타지 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Hele...,7.5,7.414835
1815,해리 포터와 비밀의 방,Chris_Columbus,판타지 액션 가족,Daniel_Radcliffe Rupert_Grint Emma_Watson Kenn...,7.4,7.328044
510,붉은 돼지,Hayao_Miyazaki,애니메이션 액션 가족 코미디 판타지 로맨스,Shûichirô_Moriyama Tokiko_Katô Sanshi_Katsura ...,7.7,7.238647
593,아르고 황금 대탐험,Don_Chaffey,판타지 액션 가족,Todd_Armstrong Nancy_Kovack Gary_Raymond Laure...,7.3,6.700119
618,타이탄족의 멸망,Desmond_Davis,판타지 액션 가족,Harry_Hamlin Judi_Bowker Burgess_Meredith Magg...,6.9,6.618700
965,신밧드의 7번째 모험,Nathan_H._Juran,판타지 액션 가족,Kerwin_Mathews Kathryn_Grant Torin_Thatcher Ri...,7.1,6.503785
3562,나니아 연대기: 캐스피언 왕자,Andrew_Adamson,판타지 액션 가족 Made in Europe,Ben_Barnes Georgie_Henley Skandar_Keynes Willi...,6.5,6.471578


In [270]:
df[df["제목"].apply(lambda x: "해리 포터" in x)][["제목", "감독"]]

,제목,감독
1735,해리 포터와 마법사의 돌,Chris_Columbus
1806,해리 포터와 아즈카반의 죄수,Alfonso_Cuarón
1815,해리 포터와 비밀의 방,Chris_Columbus
3392,해리 포터와 불의 잔,Mike_Newell
3495,해리 포터와 혼혈 왕자,David_Yates
3530,해리 포터와 불사조 기사단,David_Yates
